# 1 - Filtro dos dados dos arquivos

Para que a análise seja mais eficiente, é necessário passar um certo filtro nos dados. O objetivo é, a partir dos dois arquivos de entrada (dados educacionais e dados populacionais), gerar apenas um arquivo que contenha apenas a informação necessária. Os dados das matrículas na rede municipal de educação vieram [daqui](http://dados.prefeitura.sp.gov.br/dataset/demanda-por-vagas-educacao-infantil-e-eja). Já os populacionais vieram do site da [SEADE](http://produtos.seade.gov.br/produtos/projpop/), a partir de uma tabela nova.

In [1]:
import pandas as pd

dados_educ = pd.read_csv('dados/evolucaodemanda.csv', sep = ';')
dados_pop = pd.read_csv('dados/populacao.csv', sep = ';', encoding = 'ISO-8859-1')

Para os dados educacionais, temos que filtrar apenas as colunas que necessárias. Além disso, é preciso limpar os dados, pois algumas linhas possuem os números com um ponto (.) para indicar o milhar. Por conveniência, o mês e o ano de referência serão deixados em colunas separadas. 

In [2]:
for i in dados_educ.index:
    if not dados_educ.at[i, 'MAT_CRECHE'].is_integer():
        dados_educ.at[i, 'MAT_CRECHE'] = round(dados_educ.at[i, 'MAT_CRECHE']*1000)
    if not dados_educ.at[i, 'DEM_CRECHE'].is_integer():
        dados_educ.at[i, 'DEM_CRECHE'] = round(dados_educ.at[i, 'DEM_CRECHE']*1000)
    dados_educ.at[i, 'MES'] = dados_educ.at[i, 'MES_ANO_REF'][:3]
    dados_educ.at[i, 'ANO'] = int(dados_educ.at[i, 'MES_ANO_REF'][4:]) + 2000
       
dados_educ = dados_educ[['DISTRITO', 'MAT_CRECHE', 'DEM_CRECHE', 'MES', 'ANO', 'MES_ANO_REF']]

Para que os dados populacionais fiquem mais próximos aos educacionais, vamos alterar o nome do distrito, deixando-o sem acentos e com caixa alta. Como os dados da população estão em números inteiros, não precisaremos remover os eventuais pontos (.) que indicam o milhar. Além disso, as colunas que selecionaremos terão os mesmos nomes nas nossas duas bases de dados (educação e população).

In [3]:
dados_pop = dados_pop[['Localidades', 'Períodos', 'População de 0 a 3 Anos']]
dados_pop.columns = ['DISTRITO_F', 'ANO', 'TOTAL']
    
dados_pop['DISTRITO'] = dados_pop['DISTRITO_F'].str.normalize('NFKD')
dados_pop['DISTRITO'] = dados_pop['DISTRITO'].str.encode('ascii', errors='ignore').str.decode('utf-8')
dados_pop['DISTRITO'] = dados_pop['DISTRITO'].str.upper()

Precisamos juntar os dados, para que a análise seja feita de maneira mais eficiente. Teremos um dicionário com os seguintes valores:

- Nome do distrito (em caixa alta e sem acentos);
- Número de matriculas em creches;
- Demanda de vagas em creches;
- Mês e ano de referência;
- População do distrito na faixa etária 00 a 03 anos.

In [4]:
dados_educ_pop = dados_educ.merge(dados_pop, on = ['DISTRITO', 'ANO'])

Vamos pegar também a sigla dos distritos, para uso posterior.

In [5]:
import geopandas as gpd

distritos = gpd.read_file('dados/LAYER_DISTRITO/DEINFO_DISTRITO.shp')

for i in distritos.index:
    nome = distritos.at[i, 'NOME_DIST'].split(' ')
    if 'JD' in nome:
        nome[0] = 'JARDIM'
    if 'CID' in nome:
        nome[0] = 'CIDADE'
    distritos.at[i, 'NOME_DIST'] = ' '.join(nome)

dados_educ_pop = dados_educ_pop.merge(distritos, left_on = 'DISTRITO', right_on = 'NOME_DIST')
dados_educ_pop = dados_educ_pop[['DISTRITO', 'MAT_CRECHE', 'DEM_CRECHE', 'MES', 'ANO', 'MES_ANO_REF', 'DISTRITO_F', 'TOTAL', 'SIGLA_DIST']]

Tal dicionário será salvo num arquivo.

In [6]:
dados_educ_pop
dados_educ_pop.to_csv('dados/dados_educ_pop.csv', encoding='utf-8', index=False)

In [12]:
dados_educ_pop[dados_educ_pop['DISTRITO'] == 'PENHA']

,DISTRITO,MAT_CRECHE,DEM_CRECHE,MES,ANO,MES_ANO_REF,DISTRITO_F,TOTAL,SIGLA_DIST
2773,PENHA,447.0,623.0,jun,2006.0,jun-06,Penha,5852,PEN
2774,PENHA,464.0,769.0,set,2006.0,set-06,Penha,5852,PEN
2775,PENHA,468.0,995.0,dez,2006.0,dez-06,Penha,5852,PEN
2776,PENHA,551.0,750.0,mar,2007.0,mar-07,Penha,5723,PEN
2777,PENHA,729.0,614.0,jun,2007.0,jun-07,Penha,5723,PEN
2778,PENHA,847.0,839.0,set,2007.0,set-07,Penha,5723,PEN
2779,PENHA,854.0,386.0,dez,2007.0,dez-07,Penha,5723,PEN
2780,PENHA,1020.0,539.0,mar,2008.0,mar-08,Penha,5597,PEN
2781,PENHA,1089.0,612.0,jun,2008.0,jun-08,Penha,5597,PEN
2782,PENHA,1090.0,822.0,set,2008.0,set-08,Penha,5597,PEN
